# 1) Initialize the project setup

## 1.1) Install the python packages

In [1]:
!pip install bs4
!pip install nltk 
!pip install html5lib
!pip install pickle-mixin 
!pip install argparse 
!pip install numpy 
!pip install pandas 
!pip install scikit-learn 
!pip install xlrd
!pip install pathlib 
!pip install w3lib
!pip install svmlight

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## 1.2) Download the necessary tokenizers

In [3]:
# import the packages
import nltk
# download the tokenizers
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/tobsel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tobsel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1.3) Import the training module

In [4]:
import sys  
sys.path.append("./source")

import train

# 2) Run the experiment with some default settings

In [ ]:
train.start()

EXPERIMENT ID:  1671666016.9943871
Training it= 1 cost-factor= 1
Predicting it= 1 cost-factor= 1
Training it= 2 cost-factor= 1
Predicting it= 2 cost-factor= 1
Training it= 3 cost-factor= 1
Predicting it= 3 cost-factor= 1
Training it= 4 cost-factor= 1
Predicting it= 4 cost-factor= 1
Training it= 5 cost-factor= 1
Predicting it= 5 cost-factor= 1
The accuracy is 73.61111111111111
The f1-score is 0.7361111111111112
The credible f1-score is 0.7102486894825606
The non-credible f1-score is 0.7570346213646616
Training it= 1 cost-factor= 2
Predicting it= 1 cost-factor= 2
Training it= 2 cost-factor= 2
Predicting it= 2 cost-factor= 2
Training it= 3 cost-factor= 2
Predicting it= 3 cost-factor= 2
Training it= 4 cost-factor= 2
Predicting it= 4 cost-factor= 2
Training it= 5 cost-factor= 2
Predicting it= 5 cost-factor= 2
The accuracy is 77.77777777777777
The f1-score is 0.7305555555555555
The credible f1-score is 0.6851582014419506
The non-credible f1-score is 0.7633427783996861
Training it= 1 cost-fac

# 3) Reproduce different experiments

In [ ]:
# train parameters
dataset = "Sondhi"
features = "comm"
dump = "yes"
standard = True

In [ ]:
# start of model training
train.start(dataset, features, dump, standard)